<a href="https://colab.research.google.com/github/umersajid11/Fashion_Mnist-/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset , DataLoader
import torch.nn as nn
import torch.optim as  optim
import matplotlib.pyplot as plt

In [15]:
torch.manual_seed(42)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (f"Using Device : {device}")

Using Device : cuda


In [17]:
df = pd.read_csv('fashion-mnist_train.csv')

In [18]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#print(torch.isnan(train_dataset.features).any())

In [ ]:
#print(df.isna().sum().sum())  # total number of NaN values

In [ ]:
#print(df.isna().sum())

In [ ]:
#df = df.fillna(0)

In [19]:
x= df.iloc[:, 1:].values
y=df.iloc[: , 0].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
#Scaling the Feature
X_train = X_train/255.0
X_test = X_test/255.0

In [22]:
#Create Custom Dataset Class
class customDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, idx):
    return self.features[idx] , self.labels[idx]

In [23]:
#Create  Train Dataset Object
train_dataset = customDataset(X_train, y_train)

In [24]:
len(train_dataset)

48000

In [25]:
train_dataset[1]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [26]:
#Create Test_dataset Object
test_dataset = customDataset(X_test, y_test)

In [27]:
#DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [28]:
#no Of Batches
len (train_loader)

1500

In [30]:
#Define NN Class
class myNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )
  def forward(self, x):
    return self.model(x)

In [31]:
epochs = 25
learrning_rate = 0.001

In [32]:
model = myNN(X_train.shape[1])

#For GPU:
model = model.to(device)
#Loss
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learrning_rate)


In [33]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:
    #total_epoch_loss = 0
    #Move Data to GPU
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    #forward Pass
    outputs = model(batch_features)

    #loss
    loss = criterion(outputs, batch_labels)

    #BackPass
    optimizer.zero_grad()
    loss.backward()

    #update Grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print (f'Epoch:{epoch+1}, Loss: {avg_loss}')

Epoch:1, Loss: 0.5527251036564509
Epoch:2, Loss: 0.3926751301934322
Epoch:3, Loss: 0.3544542966733376
Epoch:4, Loss: 0.32694049732883773
Epoch:5, Loss: 0.3071295080582301
Epoch:6, Loss: 0.29175592111671966
Epoch:7, Loss: 0.27882406501223644
Epoch:8, Loss: 0.26615497599542143
Epoch:9, Loss: 0.2532227461313208
Epoch:10, Loss: 0.2466269981972873
Epoch:11, Loss: 0.23765523035948474
Epoch:12, Loss: 0.22805158472309509
Epoch:13, Loss: 0.21954751245925824
Epoch:14, Loss: 0.2133918329825004
Epoch:15, Loss: 0.20595886832848193
Epoch:16, Loss: 0.19984006962366402
Epoch:17, Loss: 0.19495809140677253
Epoch:18, Loss: 0.18768631747364997
Epoch:19, Loss: 0.18192669223745664
Epoch:20, Loss: 0.17795938213790458
Epoch:21, Loss: 0.17214797662508985
Epoch:22, Loss: 0.16649255629132192
Epoch:23, Loss: 0.16363671649247408
Epoch:24, Loss: 0.16012822507182137
Epoch:25, Loss: 0.15279452226279924


In [34]:
len(test_loader)

375

In [35]:
# Evaluation
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    correct = 0
    total = 0
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.shape[0]
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 89.27%
